## IMPORTS and UTILS

In [1]:
print("hi")

import os
import pandas as pd
import subprocess
import re
import numpy as np


hi


In [2]:
def augment_data(folder_path, new_copies=2):
    aug_folder_path = folder_path + '_AUG_' + str(new_copies)
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv.gz'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                augmented_dfs = [df]

                for copy_index in range(1, new_copies):
                    temp_df = df.copy()
                    sid_modifications = {}
                    for sid_val in temp_df['sid'].unique():
                        sid_modifications[sid_val] = np.random.choice([-1, 1])

                    temp_df['age'] = temp_df.apply(
                        lambda row: row['age'] + sid_modifications[row['sid']], axis=1
                    )
                    temp_df['sid'] = temp_df['sid'].astype(str) + f'_aug{copy_index}'
                    temp_df['mean'] = temp_df['mean'] * (
                        1 + np.random.choice([-0.01, 0.01], size=len(temp_df))
                    )

                    augmented_dfs.append(temp_df)

                final_df = pd.concat(augmented_dfs, ignore_index=True)
                relative_root = os.path.relpath(root, folder_path)
                target_root = os.path.join(aug_folder_path, relative_root)
                os.makedirs(target_root, exist_ok=True)
                output_path = os.path.join(target_root, file)
                final_df.to_csv(output_path, index=False)
augment_data("DONNES_F/COMPILATIONS", 3)
